# Projeto C318

O projeto tem como intuito fazer uma anásile de um dataset fornecido neste link do [Kaggle](https://www.kaggle.com/datasets/uciml/pima-indians-diabetes-database?resource=download&select=diabetes.csv).  
No qual, se trata de um dataset que contém alguns dados médicos, em particular, de mulheres com pelo menos 21 anos e de ascendência indígena Pima.  
Cujo o intuito é criar um modelo de ML, capaz de prever se o paciente tem ou não diabetes.

## Importação de Bibliotecas

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn import svm
from sklearn import tree
import matplotlib.pyplot as plt
from sklearn import preprocessing
from sklearn.metrics import f1_score
from sklearn.metrics import roc_curve
from imblearn.over_sampling import SMOTE
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import BernoulliNB
from sklearn.linear_model import Perceptron
from sklearn.linear_model import SGDClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_validate
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.gaussian_process import GaussianProcessClassifier


## Análise dos dados

In [ ]:
df = pd.read_csv('diabetes.csv')
df.head()

In [ ]:
print(f"Este dataset contém os dados clínicos de {len(df)} pacientes")
print(f"Features: {df.columns.to_list()[:-1]}")
print(f"Target: {df.columns.to_list()[-1]}")
print(f"Existe valores faltantes? {df.isna().any().any()}")

Diante dos tipos de dados apresentados, pode-se concluir que o dataset escolhido não apresenta variáveis categóricas

In [ ]:
df.info()

In [ ]:
df.describe()

### Análise da Variável Target

A variável target é qualitativa(categórica), logo estamos trabalhando com um modelo de classificação.  
Na figura abaixo podemos analisar a quantidade de pacientes com(1) e sem(0) diabetes.  
É possível notar que o dataset se encontra desbalanceado.

In [ ]:
plt.pie(df['Outcome'].value_counts().to_list(), autopct='%.02f%%')
plt.legend(['0', '1'])
plt.show()

### Divisão dos Dados para Treino e Teste

Dado o desbalanceamento dos dados, a divisão dos dados usando o método [train_test_split](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html) do sklearn, iria inserir uma inconsistência nos dados. Podendo coletar mais amostras de uma população do que de outra. Logo, o método adequado é o [StratifiedShuffleSplit](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.StratifiedShuffleSplit.html).

In [ ]:
split = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42)
for train_index, test_index in split.split(df, df.Outcome):
    strat_train_set = df.loc[train_index]
    strat_test_set = df.loc[test_index]

### Análise das Features

#### Pregnancies

Representa o número de vezes que aquela paciente ficou grávida.  
Como podemos analisar abaixo, este valor varia entre 0 e 17.  
Neste gráfico podemos analisar a proporção de pacientes com e sem diabetes, levando em consideração o número de gestações.  
Uma curiosidade é que cerca de 140 pacientes, tiveram apenas 1 filho(a). E aproximadamente 30 delas apresentaram diabetes.

In [ ]:
aux = strat_train_set[['Pregnancies','Outcome']].value_counts().sort_index().reset_index()
aux.columns = ['N° Gestacao/Pessoa', 'Tem Diabetes', 'N° de Pacientes']
plt.figure(figsize=(8, 6))
sns.barplot(aux, x='N° Gestacao/Pessoa', y='N° de Pacientes', hue='Tem Diabetes')
plt.show()

In [ ]:
ax = sns.displot(strat_train_set, x="Pregnancies", hue='Outcome')
plt.xlabel("Pregnancies")
plt.ylabel('N° de Pacientes')
plt.show()

#### Glucose

Esta feature representa a "Concentração de glicose plasmática a 2 horas em um teste oral de tolerância à glicose".  
Neste teste é administrado uma quantidade X de glicose no paciente e após 2 horas é medido o valor de glicemia.  
Valores entre 140 e 200 mg/dL é considerado pré-diabetes e acima de 200mg/dL confirma o diagnóstico de diabetes.  
No gráfico abaixo, podemos observar tal feito, o número de casos sem diabetes com valores acima de 150mg/dL diminui considerávelmente.

In [ ]:
ax = sns.displot(strat_train_set, x="Glucose", hue='Outcome')
plt.xlabel("Glicose")
plt.ylabel('N° de Pacientes')
plt.show()

#### BloodPressure

Pressão arterial diastólica (mmHg)

In [ ]:
sns.displot(strat_train_set, x="BloodPressure", hue='Outcome')
plt.xlabel("Pressão arterial diastólica (mmHg)")
plt.ylabel('N° de Pacientes')
plt.show()

#### SkinThickness

Espessura da dobra cutânea do tríceps (mm)

In [ ]:
sns.displot(strat_train_set, x="SkinThickness", hue='Outcome')
plt.xlabel("Espessura da pele (mm)")
plt.ylabel('N° de Pacientes')
plt.show()

#### Insulin

Representa a "Insulina sérica de 2 horas (mu U/ml)".

In [ ]:
sns.displot(strat_train_set, x="Insulin", hue='Outcome')
plt.xlabel("Insulina (mu U/ml)")
plt.ylabel('N° de Pacientes')
plt.show()

#### BMI

Índice de massa corporal (peso em kg/(altura em m)^2)

In [ ]:
sns.displot(strat_train_set, x="BMI", hue='Outcome')
plt.xlabel("Índice de massa corporal")
plt.ylabel('N° de Pacientes')
plt.show()

#### DiabetesPedigreeFunction

Função hereditária do diabetes

In [ ]:
sns.displot(strat_train_set, x="DiabetesPedigreeFunction", hue='Outcome')
plt.xlabel("Relação Hereditária")
plt.ylabel('N° de Pacientes')
plt.show()

#### Age

Idade dos pacientes.

In [ ]:
sns.displot(strat_train_set, x="Age", hue='Outcome')
plt.xlabel("Idade")
plt.ylabel('N° de Pacientes')
plt.show()

### Matriz de Correlação

In [ ]:
plt.figure(figsize=(8, 6))
print(f"Matriz de correlação das variáveis:\n{sns.heatmap(strat_train_set.corr(), annot=True)}")
plt.show()

### Feature Importance dos Dados Desbalanceados

In [ ]:
clf = RandomForestClassifier()
clf = clf.fit(strat_train_set.drop('Outcome', axis=1).values, strat_train_set.Outcome.values)
feature_importance = pd.Series(clf.feature_importances_, index=list(strat_train_set.drop('Outcome', axis=1).columns))
feature_importance = feature_importance.sort_values()
feature_importance.plot(kind='barh')

### Modelos

In [ ]:
models = {
    'DecisionTreeClassifier': tree.DecisionTreeClassifier(), 
    'SGDClassifier': SGDClassifier(), 
    'RandomForestClassifier': RandomForestClassifier(), 
    'LogisticRegression': LogisticRegression(),
    'SVC': svm.SVC(),
    'KNeighborsClassifier': KNeighborsClassifier(),
    'GaussianNB': GaussianNB(),
    'AdaBoostClassifier': AdaBoostClassifier(),
    'MLPClassifier': MLPClassifier(),
    'Perceptron': Perceptron(),
    'GaussianProcessClassifier': GaussianProcessClassifier(),
    'BernoulliNB': BernoulliNB(),
    'GradientBoostingClassifier': GradientBoostingClassifier()
    }   

### Cross-validation sem Balanceamento dos Dados

In [ ]:
count = 0
results = f'Modelo;Accuracy_{count};Mean Accuracy_{count};STD Accuracy_{count};F1_{count};Mean F1_{count};STD F1_{count};Precision_{count};Mean Precision_{count};STD Precision_{count};Recall_{count};Mean Recall_{count};STD Recall_{count}\n'
count += 1
for name, clf in models.items():
    X = strat_train_set.drop('Outcome', axis=1).values
    y = strat_train_set.Outcome.copy().values
    if name == 'LogisticRegression':
        X = preprocessing.StandardScaler().fit(X).transform(X)

    clf = clf.fit(X, y)
    cross = cross_validate(clf, X, y, scoring=["accuracy", "f1", "precision", "recall"], cv=3)
    results += f"{name};{list(cross['test_accuracy'])};{cross['test_accuracy'].mean()};{cross['test_accuracy'].std()}"
    results +=       f";{list(cross['test_f1'])};{cross['test_f1'].mean()};{cross['test_f1'].std()}"
    results +=       f";{list(cross['test_precision'])};{cross['test_precision'].mean()};{cross['test_precision'].std()}"
    results +=       f";{list(cross['test_recall'])};{cross['test_recall'].mean()};{cross['test_recall'].std()}\n"

with open('results.csv', 'w', encoding='utf-8') as w:
    w.write(results)

In [ ]:
results = pd.read_csv('results.csv', delimiter=';')
results[['Modelo', 'Mean Accuracy_0', 'Mean F1_0', 'Mean Precision_0', 'Mean Recall_0']].plot(kind='bar', x='Modelo', figsize=(15,10), rot=50)

### Cross-validation Com Balanceamento dos Dados

In [ ]:
sm = SMOTE(random_state=42)
X_smote, y_smote = sm.fit_resample(X, y)

In [ ]:
results = f'Modelo;Accuracy_{count};Mean Accuracy_{count};STD Accuracy_{count};F1_{count};Mean F1_{count};STD F1_{count};Precision_{count};Mean Precision_{count};STD Precision_{count};Recall_{count};Mean Recall_{count};STD Recall_{count}\n'
count += 1
for name, clf in models.items():
    X = X_smote
    y = y_smote
    if name == 'LogisticRegression':
        X = preprocessing.StandardScaler().fit(X).transform(X)

    clf = clf.fit(X, y)
    cross = cross_validate(clf, X, y, scoring=["accuracy", "f1", "precision", "recall"], cv=3)
    results += f"{name};{list(cross['test_accuracy'])};{cross['test_accuracy'].mean()};{cross['test_accuracy'].std()}"
    results +=       f";{list(cross['test_f1'])};{cross['test_f1'].mean()};{cross['test_f1'].std()}"
    results +=       f";{list(cross['test_precision'])};{cross['test_precision'].mean()};{cross['test_precision'].std()}"
    results +=       f";{list(cross['test_recall'])};{cross['test_recall'].mean()};{cross['test_recall'].std()}\n"

with open('results_SMOTE.csv', 'w', encoding='utf-8') as w:
    w.write(results)

In [ ]:
results = pd.read_csv('results.csv', delimiter=';')
results_SMOTE = pd.read_csv('results_SMOTE.csv', delimiter=';')
r = pd.merge(results, results_SMOTE, how='left', on='Modelo')
r[['Modelo', 'Mean F1_0', 'Mean F1_1']].plot(kind='bar', x='Modelo', figsize=(15,10), rot=50)

Como pode ser observado, todos os modelos melhoram após balancear os dados.

In [ ]:
X_smote_stand = preprocessing.StandardScaler().fit(X_smote).transform(X_smote)

In [ ]:
results = f'Modelo;Accuracy_{count};Mean Accuracy_{count};STD Accuracy_{count};F1_{count};Mean F1_{count};STD F1_{count};Precision_{count};Mean Precision_{count};STD Precision_{count};Recall_{count};Mean Recall_{count};STD Recall_{count}\n'
count += 1
for name, clf in models.items():
    clf = clf.fit(X_smote_stand, y_smote)
    cross = cross_validate(clf, X_smote_stand, y_smote, scoring=["accuracy", "f1", "precision", "recall", "roc_auc"], cv=3)
    results += f"{name};{list(cross['test_accuracy'])};{cross['test_accuracy'].mean()};{cross['test_accuracy'].std()}"
    results +=       f";{list(cross['test_f1'])};{cross['test_f1'].mean()};{cross['test_f1'].std()}"
    results +=       f";{list(cross['test_precision'])};{cross['test_precision'].mean()};{cross['test_precision'].std()}"
    results +=       f";{list(cross['test_recall'])};{cross['test_recall'].mean()};{cross['test_recall'].std()}\n"

with open('results_SMOTE_STAND.csv', 'w', encoding='utf-8') as w:
    w.write(results)

In [ ]:
results = pd.read_csv('results.csv', delimiter=';')
results_SMOTE = pd.read_csv('results_SMOTE.csv', delimiter=';')
results_SMOTE_STAND = pd.read_csv('results_SMOTE_STAND.csv', delimiter=';')
r = pd.merge(results, results_SMOTE, how='left', on='Modelo')
r = pd.merge(r, results_SMOTE_STAND, how='left', on='Modelo')
r[['Modelo', 'Mean F1_0', 'Mean F1_1', 'Mean F1_2']].plot(kind='bar', x='Modelo', figsize=(15,10), rot=50)

Como podemos analisar acima, a maioria dos modelos apresentaram uma melhora após aplicar o [Standard Scaler](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html#sklearn.preprocessing.StandardScaler)

### Predição do Modelo

In [ ]:
models = {
    'RandomForestClassifier': RandomForestClassifier(), 
    'SVC': svm.SVC(),
    'GaussianProcessClassifier': GaussianProcessClassifier(),
}   

In [ ]:
X_test = strat_test_set.drop('Outcome', axis=1).values
y_test = strat_test_set.Outcome.copy().values

In [ ]:
X_test = preprocessing.StandardScaler().fit(X_test).transform(X_test)

In [ ]:
sm = SMOTE(random_state=42)
X_test_smote, y_test_smote = sm.fit_resample(X_test, y_test)
print(X_smote.shape)

In [ ]:
results = f'Modelo;F1\n'
count += 1
for name, clf in models.items():
    clf = clf.fit(X_smote_stand, y_smote)
    y_pred = clf.predict(X_test_smote)
    results += f'{name};{f1_score(y_true=y_test_smote, y_pred=y_pred)}\n'

with open('results_predict.csv', 'w', encoding='utf-8') as w:
    w.write(results)

In [ ]:
results = pd.read_csv('results_predict.csv', delimiter=';')
results.plot(kind='bar', x='Modelo', figsize=(8,8), rot=0)